# Clasificador de correos spam con Naive Bayes

Vamos a crear un clasificador de correos de spam usando el algoritmo de Naive Bayes. Para esto usaremos mensajes de texto SMS recopilados por Tiago A. Almeida y José María Gómez. Estos datos se pueden descargar del [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). 

## Leer los datos


In [1]:
import pandas as pd

In [2]:
sms = pd.read_csv('SMSSpamCollection', header=None, sep='\t', names=['Label', 'SMS'])
sms.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
sms.shape

(5572, 2)

El archivo tiene 5574 registros de mensajes de texto. Miremos la distribución de la clasificación.

In [4]:
freq = sms['Label'].value_counts(normalize=True) * 100
freq

ham     86.593683
spam    13.406317
Name: Label, dtype: float64

El ~86.6% de los mensajes no son spam y el restante 13.4% sí lo es. Esto parece estar alineado al hecho de que la mayoría de los mensajes que la gente recibe no son spam.

## Sets de entreno y testeo
Vamos a destinar 80% de los registros a un set de entreno y 20% para testear el modelo.

In [5]:
# Mezclar los datos
shuffle = sms.sample(frac=1, random_state=1)

# Índice de 80%
train_index = round(len(shuffle) * 0.8)

# Split
train = shuffle[:train_index].reset_index(drop=True)
test = shuffle[train_index:].reset_index(drop=True)

print(train.shape)
print(test.shape)

(4458, 2)
(1114, 2)


Vamos a calcular en cada set la distribución de los correos según el label, se espera que se parezca al total de los datos de ~87% ham y ~13% spam.

In [6]:
train['Label'].value_counts(normalize=True) * 100

ham     86.54105
spam    13.45895
Name: Label, dtype: float64

In [7]:
test['Label'].value_counts(normalize=True) * 100

ham     86.804309
spam    13.195691
Name: Label, dtype: float64

Ambos sets tienen distribuciones adecuadas.

## Limpiando los datos
Vamos a cambiar la forma en la que están los datos para facilitar los cálculos más adelante. El principal cambio que se quiere hacer es deshacernos de la columna `SMS` y en cambio generar columnas para cada palabra con el fin de identificar en una columna la frecuencia de cada palabra dentro del mensaje. 

Aquí se ve un ejemplo de este cambio:

| label | SMS |
|-|-|
|spam|FREE entry! claim free prize|

|label|free|entry|claim|prize|
|-|-|-|-|-|
|spam|2|1|1|1|

Las palabras se pasan a minúscula y se ignoran las puntuaciones. La columna de cada palabra cuenta el número de veces que esa palabra ocurre dentro del mensaje.

## Puntuación y minúsculas
Empezamos retirando la puntuación y cambiando las palabras a minúsculas.

In [8]:
# Antes
train.head()

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...


In [9]:
# Después
train['SMS'] = train['SMS'].str.replace('\W', ' ')
train['SMS'] = train['SMS'].str.lower()
train.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


## Crear el vocabulario
Para poder crear las columnas de todas las palabras debemos conocer todas las plabras que se encuentran en este set de mensajes.

In [10]:
train['SMS'] = train['SMS'].str.split()

vocabulary = []

for sms in train['SMS']:
    for word in sms:
        vocabulary.append(word)
        
vocabulary = list(set(vocabulary))

In [11]:
print('Hay {} palabras únicas en el set de entreno.'.format(len(vocabulary)))

Hay 7783 palabras únicas en el set de entreno.


## Columnas por palabra
Primero vamos a crear un diccionario con las palabras del vocabulario como llaves. Luego, haremos un loop sobre los mensajes para poblar las frecuencias de cada una de estas palabras sobre el diccionario.

In [12]:
# A cada palabra le da una lista con el número de entradas como mensajes
word_count_per_sms = {word: [0] * len(train['SMS']) for word in vocabulary}

for index, sms in enumerate(train['SMS']):
    for word in sms:
        word_count_per_sms[word][index] += 1

In [13]:
word_count = pd.DataFrame(word_count_per_sms)
word_count.head()

,0,00,000,000pes,008704050406,0089,01223585334,02,0207,02072069400,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


In [14]:
train_clean = pd.concat([train, word_count], axis=1)
train_clean.head()

,Label,SMS,0,00,000,000pes,008704050406,0089,01223585334,02,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


## Constantes en Naive Bayes
El algoritmo está interesado en calcular aproximaciones para las probablidades condicionales de spam dado el mensaje. Como tal:

$$\Pr\left(\text{spam}\big|w_1,\dots,w_n\right)\propto\Pr(\text{spam})\prod_{i=1}^n\Pr\left(w_i\big|\text{spam}\right)\\
\Pr\left(\text{ham}\big|w_1,\dots,w_n\right)\propto\Pr(\text{ham})\prod_{i=1}^n\Pr\left(w_i\big|\text{ham}\right)$$

Usando las siguientes aproximaciones:

$$\Pr\left(w_i\big|\text{spam}\right) = \frac{N_{w_i|\text{ spam}}+\alpha}{N_{\text{spam}}+\alpha N_{\text{vocabulary}}}\\
\Pr\left(w_i\big|\text{ham}\right) = \frac{N_{w_i|\text{ ham}}+\alpha}{N_{\text{ham}}+\alpha N_{\text{vocabulary}}}
$$

Donde:

$\Pr(\text{spam})$ es la proporción de mensajes etiquetados como spam.

$\Pr(\text{ham})$ es la proporción de mensajes etiquetados como ham.

$N_{\text{spam}}$ es igual al número total de palabras (con repeticiones) dentro de los mensajes spam.

$N_{\text{ham}}$ es igual al número total de palabras (con repeticiones) dentro de los mensajes ham.

$N_{\text{vocabulary}}$ es el número de plabras en el vocabulario.

$\alpha = 1$ es el parámetro de smoothing en el Laplace smoothing que se está usando.

Todas estas cantidades son fijas sin importar el mensaje que se escoja, de manera que primero vamos a calcular esto.

In [15]:
# Separar spam y ham
spam = train_clean[train_clean['Label'] == 'spam']
ham = train_clean[train_clean['Label'] == 'ham']

# P(spam), P(ham)
p_spam = len(spam) / len(train_clean)
p_ham = len(ham) / len(train_clean)

# N_spam, N_ham
n_spam = spam['SMS'].apply(len).sum()
n_ham = ham['SMS'].apply(len).sum()

# N_voc
n_voc = len(vocabulary)

# Laplace smoothing
alpha = 1

## Calcular parámetros
Ya habiendo inicializado los anteriores términos constantes, seguimos a calcular los parámetros para cada palabra $w_i$ del vocabulario, a saber, se deben calcular ambas

$$
\Pr\left(w_i\big|\text{spam}\right) \\
\Pr\left(w_i\big|\text{ham}\right)
$$

Usando las fórmulas que se mostraron anteriormente.

In [16]:
# Inicialización de parámetros
params_spam = {word: 0 for word in vocabulary}
params_ham = {word: 0 for word in vocabulary}

# Cálculo de los parámetros
for word in vocabulary:
    # Calcula el número de veces (con repeticiones) que aparece
    # la palabra dentro de los mensajes de spam
    n_word_given_spam = spam[word].sum()
    params_spam[word] = (n_word_given_spam + alpha) / (n_spam + alpha * n_voc)
    
    # Calcula el número de veces (con repeticiones) que aparece
    # la palabra dentro de los mensajes de ham
    n_word_given_ham = ham[word].sum()
    params_ham[word] = (n_word_given_ham + alpha) / (n_ham + alpha * n_voc)

## Clasificando nuevos mensajes
El clasificador obtiene un mensaje $(w_1,\dots,w_n)$.

Calcula $\Pr\left(\text{spam}\big|w_1,\dots,w_n\right)$ y $\Pr\left(\text{ham}\big|w_1,\dots,w_n\right)$.

Si $\Pr\left(\text{spam}\big|w_1,\dots,w_n\right) > \Pr\left(\text{ham}\big|w_1,\dots,w_n\right)$ el mensaje se considera spam.

Si $\Pr\left(\text{spam}\big|w_1,\dots,w_n\right) < \Pr\left(\text{ham}\big|w_1,\dots,w_n\right)$ el mensaje se considera ham.

Si hay igualdad se pide ayuda a ser clasificado por un humano.

In [17]:
import re

def classify(message):
    message = re.sub('\W', ' ', message)
    message = message.lower().split()
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in params_spam:
            p_spam_given_message *= params_spam[word]
            
        if word in params_ham:
            p_ham_given_message *= params_ham[word]
    print('Pr(spam|message): ', p_spam_given_message)
    print('Pr(Ham|message): ', p_ham_given_message)   
    
    if p_spam_given_message > p_ham_given_message:
        print('Label: Spam')
    elif p_spam_given_message < p_ham_given_message:
        print('Label: Ham')
    else:
        print('Equal probabilities, human help needed')

Miremos el clasificador con dos ejemplos

In [18]:
classify('WINNER!! secret code to unlock money: C3421.')

Pr(spam|message):  1.546355055003161e-16
Pr(Ham|message):  3.4155391060244023e-19
Label: Spam


In [19]:
classify('Its okay, see you later')

Pr(spam|message):  3.0154394734979374e-19
Pr(Ham|message):  5.703376196223765e-14
Label: Ham


## Midiendo la precisión del clasificador
Vamos a testear el clasificador en los 1114 mensajes del test set. Primero vamos a cambiar la función de clasificación para que retorne el label.

In [20]:
def test_classify(message):
    message = re.sub('\W', ' ', message)
    message = message.lower().split()
    
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    for word in message:
        if word in params_spam:
            p_spam_given_message *= params_spam[word]
            
        if word in params_ham:
            p_ham_given_message *= params_ham[word] 
    
    if p_spam_given_message > p_ham_given_message:
        return 'spam'
    elif p_spam_given_message < p_ham_given_message:
        return 'ham'
    else:
        return 'help needed'

Ahora usamos esta función para crear una nueva columna en el test set con los valores que predice el algoritmo.

In [21]:
test['predicted'] = test['SMS'].apply(test_classify)
test.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


Para medir el desempeño del clasificador vamos a usar la métrica __accuracy__:

$$\text{accuracy}=\frac{\text{número de mensajes bien clasificados}}{\text{número total de mensajes clasifiados}}$$

In [22]:
correct = (test['Label'] == test['predicted']).sum()
total = test.shape[0]
accuracy = correct / total

print('Correctos:', correct)
print('Total:', total)
print('Accuracy:', accuracy)

Correctos: 1100
Total: 1114
Accuracy: 0.9874326750448833


## Conclusión
El __accuracy__ es del ~98.74%. Lo cual es muy bueno considerando que los 1114 mensajes no se habían visto al entrenar. 

Una posible consideración para posiblemente hacer el clasificador más riguroso podría ser permitir el uso de mayúsculas dentro de los mensajes.